In [1]:
import theano
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Conv1D, MaxPooling1D, ZeroPadding1D
from keras.optimizers import SGD
import numpy as np
import os

Using Theano backend.


In [23]:
feat_0_path='data/mon_exp/feat/0'
feat_1_path='data/mon_exp/feat/1'
max_feat_count=5000

feats_0_list = []
for fil in os.listdir(feat_0_path):
    feats = []
    with open(os.path.join(feat_0_path, fil)) as f:
        feats = f.readlines()
        feats = [float(line) for line in feats]
    feats = np.array(feats)
    feats1 = np.zeros(max_feat_count)
    length = min(len(feats), max_feat_count)
    feats1[:length] = feats[:length]
    #print len(feats)
    feats_0_list.append(feats1)

feats_1_list = []
for fil in os.listdir(feat_1_path):
    feats = []
    with open(os.path.join(feat_1_path, fil)) as f:
        feats = f.readlines()
        feats = [float(line) for line in feats]
    feats = np.array(feats)
    feats1 = np.zeros(max_feat_count)
    length = min(len(feats), max_feat_count)
    feats1[:length] = feats[:length]
    #print len(feats1)
    feats_1_list.append(feats1)
  

In [24]:
num_test=100
f0 = np.array(feats_0_list)
f1 = np.array(feats_1_list)
print f0.shape
print f1.shape
f0_tr = f0[:-num_test, :]
f1_tr = f1[:-num_test, :]
print f0_tr.shape
print f1_tr.shape
f0_ts = f0[-num_test:, :]
f1_ts = f1[-num_test:, :]
print f0_ts.shape
print f1_ts.shape


(2815, 5000)
(320, 5000)
(2715, 5000)
(220, 5000)
(100, 5000)
(100, 5000)


In [25]:
lab_0_tr = np.zeros((f0_tr.shape[0],2))
lab_1_tr = np.zeros((f1_tr.shape[0],2))
lab_0_ts = np.zeros((f0_ts.shape[0],2))
lab_1_ts = np.zeros((f1_ts.shape[0],2))

lab_0_tr[:,0] = 1
lab_1_tr[:,1] = 1
lab_0_ts[:,0] = 1
lab_1_ts[:,1] = 1

print lab_0_tr.shape
x_0_tr = np.append(f0_tr,lab_0_tr, axis=1)
x_1_tr = np.append(f1_tr, lab_1_tr, axis=1)
print x_0_tr.shape
x_0_ts = np.append(f0_ts, lab_0_ts, axis=1)
x_1_ts = np.append(f1_ts, lab_1_ts, axis=1)

(2715, 2)
(2715, 5002)


In [26]:
x_tr = np.vstack([x_0_tr, x_1_tr])
print x_tr.shape
x_ts = np.vstack([x_0_ts, x_1_ts])
np.random.shuffle(x_tr)
np.random.shuffle(x_ts)

(2935, 5002)


In [36]:
X_tr = x_tr[:, 0:max_feat_count]
y_tr = x_tr[:, max_feat_count:]
X_ts = x_ts[:, 0:max_feat_count]
y_ts = x_ts[:, max_feat_count:]
print X_tr.shape
print y_tr.shape
print X_ts.shape
print y_ts.shape

(2935, 5000)
(2935, 2)
(200, 5000)
(200, 2)


In [28]:
def VGG_16(max_len, weights_path=None):
    model = Sequential()
    model.add(ZeroPadding1D((1,1),input_shape=(1, max_len)))
    model.add(Conv1D(256, (3, ), activation='relu'))
    model.add(ZeroPadding1D((1,1)))
    model.add(Conv1D(256, (3, ), activation='relu'))
    #model.add(MaxPooling2D((2,2), strides=(2,2)))
    print model.output_shape
    model.add(ZeroPadding1D((1,1)))
    model.add(Conv1D(128, (3, ), activation='relu'))
    model.add(ZeroPadding1D((1,1)))
    model.add(Conv1D(128, (3, ), activation='relu'))
    #print "Before max pool %s" %str(model.output_shape)
    #model.add(MaxPooling1D(pool_size=1, strides=1))
    #print model.output_shape
    model.add(ZeroPadding1D((1,1)))
    model.add(Conv1D(64, (3, ), activation='relu'))
    model.add(ZeroPadding1D((1,1)))
    model.add(Conv1D(64, (3, ), activation='relu'))
    model.add(ZeroPadding1D((1,1)))
    model.add(Conv1D(64, (3, ), activation='relu'))
    #print model.output_shape
    #model.add(MaxPooling1D(2, strides=2))
    #print model.output_shape
    model.add(ZeroPadding1D((1,1)))
    model.add(Conv1D(32, (3, ), activation='relu'))
    model.add(ZeroPadding1D((1,1)))
    model.add(Conv1D(32, (3, ), activation='relu'))
    model.add(ZeroPadding1D((1,1)))
    model.add(Conv1D(32, (3, ), activation='relu'))
    #model.add(MaxPooling1D(2, strides=2))
    #print model.output_shape
    '''
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    '''
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    '''model.add(Dense(16, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(8, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='relu'))
    model.add(Dropout(0.5))'''
    
    
    if weights_path:
        model.load_weights(weights_path)

    return model


In [29]:
print theano.__version__
import keras.backend as K
K.set_image_dim_ordering('th')
print K.image_dim_ordering()
input_shape=(1,max_feat_count)
print all(input_shape[1:])
from theano import ifelse
print theano.ifelse.ifelse

0.10.0dev1.dev-d51233517debf3a1231a33058d65e8f969db923d
th
True
<function ifelse at 0x7f0755fb4230>


In [37]:
model = Sequential()
#model.add(Dense(2, activation='relu', input_dim=max_feat_count))
#model.add(Dense(2, activation='softmax' ))
model.add(Dense(2, activation='softmax',input_dim=max_feat_count ))

In [38]:
#model = VGG_16(max_feat_count)
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy']  )


In [31]:
X_tr = X_tr.reshape(X_tr.shape[0], 1, X_tr.shape[1])
X_ts = X_ts.reshape(X_ts.shape[0], 1, X_ts.shape[1])

In [32]:
print X_tr.shape
print y_tr.shape
print model.summary()

(2935, 1, 5000)
(2935, 2)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding1d_11 (ZeroPaddi (None, 3, 5000)           0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 1, 256)            3840256   
_________________________________________________________________
zero_padding1d_12 (ZeroPaddi (None, 3, 256)            0         
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 1, 256)            196864    
_________________________________________________________________
zero_padding1d_13 (ZeroPaddi (None, 3, 256)            0         
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 1, 128)            98432     
_________________________________________________________________
zero_padding1d_14 (ZeroPaddi (None, 3, 128)       

In [39]:
model.fit(X_tr, y_tr, batch_size=32, epochs=10)

Epoch 1/10
2935/2935 [==============================] - 0s - loss: 0.4636 - acc: 0.8528     
Epoch 2/10
2935/2935 [==============================] - 0s - loss: 0.3136 - acc: 0.8920     
Epoch 3/10
2935/2935 [==============================] - 0s - loss: 0.2461 - acc: 0.9233     
Epoch 4/10
2935/2935 [==============================] - 0s - loss: 0.2144 - acc: 0.9339     
Epoch 5/10
2935/2935 [==============================] - 0s - loss: 0.1892 - acc: 0.9417     
Epoch 6/10
2935/2935 [==============================] - 0s - loss: 0.1789 - acc: 0.9434     
Epoch 7/10
2935/2935 [==============================] - 0s - loss: 0.1695 - acc: 0.9472     
Epoch 8/10
2935/2935 [==============================] - 0s - loss: 0.1650 - acc: 0.9479     
Epoch 9/10
2935/2935 [==============================] - 0s - loss: 0.1602 - acc: 0.9509     
Epoch 10/10
2935/2935 [==============================] - 0s - loss: 0.1564 - acc: 0.9523     


In [40]:
model.evaluate(X_ts, y_ts, batch_size=200)

200/200 [==============================] - 0s


[1.3019599702099611, 0.58499999999999996]

In [41]:
print model.predict(X_ts[0:20])

[[  6.52191876e-01   3.47808124e-01]
 [  9.20623049e-01   7.93769511e-02]
 [  9.70065467e-01   2.99345334e-02]
 [  9.99934187e-01   6.58130605e-05]
 [  9.75846915e-01   2.41530847e-02]
 [  8.01496424e-01   1.98503576e-01]
 [  9.39989632e-01   6.00103680e-02]
 [  9.99933591e-01   6.64089054e-05]
 [  9.28109464e-01   7.18905357e-02]
 [  9.41305200e-01   5.86948002e-02]
 [  8.93253593e-01   1.06746407e-01]
 [  3.54454213e-01   6.45545787e-01]
 [  9.15221365e-01   8.47786351e-02]
 [  8.65772242e-01   1.34227758e-01]
 [  9.19756560e-01   8.02434402e-02]
 [  9.99613721e-01   3.86278909e-04]
 [  9.41305200e-01   5.86948002e-02]
 [  1.00000000e+00   2.56811587e-10]
 [  9.99207900e-01   7.92100171e-04]
 [  9.39352941e-01   6.06470595e-02]]


In [24]:
print model.predict(X_ts[0:20])

[[  9.99999944e-01   5.58455853e-08]
 [  1.00000000e+00   2.99722810e-10]
 [  5.06029170e-01   4.93970830e-01]
 [  5.06029170e-01   4.93970830e-01]
 [  9.99999868e-01   1.32425045e-07]
 [  9.99999980e-01   1.99602362e-08]
 [  5.06029170e-01   4.93970830e-01]
 [  5.06029170e-01   4.93970830e-01]
 [  5.06029170e-01   4.93970830e-01]
 [  5.06029170e-01   4.93970830e-01]
 [  5.07436490e-01   4.92563510e-01]
 [  5.06027682e-01   4.93972318e-01]
 [  5.06029170e-01   4.93970830e-01]
 [  5.06029170e-01   4.93970830e-01]
 [  9.99999999e-01   1.17829729e-09]
 [  1.00000000e+00   3.32733742e-10]
 [  5.06029170e-01   4.93970830e-01]
 [  5.06029170e-01   4.93970830e-01]
 [  5.06028193e-01   4.93971807e-01]
 [  9.99999980e-01   1.99602362e-08]]


In [10]:
print X_ts[0:20,0:20]

[[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
   1.  1.]
 [ 1. -1. -1.  1.  1. -1. -1.  1.  1.  1. -1.  1.  1. -1.  1.  1.  1.  1.
   1.  1.]
 [ 1. -1. -1.  1.  1.  1. -1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.
   1.  1.]
 [ 1. -1. -1.  1.  1. -1.  1. -1.  1.  1. -1.  1. -1. -1.  1.  1.  1.  1.
   1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
   1.  1.]
 [ 1. -1. -1.  1.  1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
   1.  1.]
 [-1. -1.  1.  1. -1.  1.  1. -1.  1. -1.  1.  1.  1. -1.  1.  1. -1.  1.
   1.  1.]
 [ 1. -1.  1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1. -1.  1.  1.  1.
   1.  1.]
 [ 1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.
  -1.  1.]
 [ 1.  1. -1. -1.  1.  1. -1.  1.  1.  1.  1. -1. -1. -1.  1.  1. -1.  1.
   1. -1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
   1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.

In [11]:
print y_ts[0:20]

[[ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]]
